# Preprocesar

- El objetivo de esta notebook es procesar la informacion para su posterior analisis y modelado.

In [1]:
# librerias necesarias
import pandas as pd
import numpy as np

In [2]:
# levantamos el set de datos ubicado en la carpeta "data" del proyecto
df = pd.read_csv("../data/challenge_edMachina.csv", sep=";")
df.head()

,particion,periodo,nota_final_materia,fecha_mesa_epoch,nombre_examen,nota_parcial,assignment_id,ass_name,ass_created_at,ass_due_at,...,sub_uuid,score,submission_type,s_submitted_at,s_graded_at,s_created_at,user_uuid,course_uuid,course_name,legajo
0,0,1-2022,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13df535e-065c-4593-98ea-5b1e29015b7d,09614210-fce2-48bc-93e3-bc4bd441fe00,Progressive homogeneous structure,834066QFF
1,1,1-2022,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13df535e-065c-4593-98ea-5b1e29015b7d,09614210-fce2-48bc-93e3-bc4bd441fe00,Progressive homogeneous structure,834066QFF
2,2,1-2022,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13df535e-065c-4593-98ea-5b1e29015b7d,09614210-fce2-48bc-93e3-bc4bd441fe00,Progressive homogeneous structure,834066QFF
3,3,1-2022,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13df535e-065c-4593-98ea-5b1e29015b7d,09614210-fce2-48bc-93e3-bc4bd441fe00,Progressive homogeneous structure,834066QFF
4,4,1-2022,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13df535e-065c-4593-98ea-5b1e29015b7d,09614210-fce2-48bc-93e3-bc4bd441fe00,Progressive homogeneous structure,834066QFF


- Con un breve vistazo podemos observar una gran cantidad de datos faltantes

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196124 entries, 0 to 196123
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   particion           196124 non-null  int64  
 1   periodo             196124 non-null  object 
 2   nota_final_materia  196124 non-null  float64
 3   fecha_mesa_epoch    7010 non-null    float64
 4   nombre_examen       7010 non-null    object 
 5   nota_parcial        7010 non-null    float64
 6   assignment_id       26061 non-null   float64
 7   ass_name            26061 non-null   object 
 8   ass_created_at      26061 non-null   float64
 9   ass_due_at          26061 non-null   float64
 10  ass_unlock_at       24760 non-null   float64
 11  ass_lock_at         25979 non-null   float64
 12  points_possible     26061 non-null   float64
 13  ass_name_sub        24206 non-null   object 
 14  sub_uuid            24206 non-null   object 
 15  score               23876 non-null

In [4]:
df.isnull().sum()

particion                  0
periodo                    0
nota_final_materia         0
fecha_mesa_epoch      189114
nombre_examen         189114
nota_parcial          189114
assignment_id         170063
ass_name              170063
ass_created_at        170063
ass_due_at            170063
ass_unlock_at         171364
ass_lock_at           170145
points_possible       170063
ass_name_sub          171918
sub_uuid              171918
score                 172248
submission_type       171918
s_submitted_at        171918
s_graded_at           172247
s_created_at          171918
user_uuid                  0
course_uuid                0
course_name                0
legajo                     0
dtype: int64

- Acomodamos fechas a un formato mas descriptivo

In [5]:
date_columns = ["ass_created_at","ass_due_at","ass_unlock_at","ass_lock_at","fecha_mesa_epoch","s_submitted_at","s_graded_at","s_created_at"]

for column in date_columns:
    df[column] = pd.to_datetime(df[column], unit="s", errors="coerce")

- La columna periodo presenta una pequeña inconsistencia y es corregida

In [6]:
df.periodo.unique()

array(['1-2022', '01-2022', '2-2022'], dtype=object)

In [7]:
df["periodo"] = df["periodo"].astype(str).str.replace("01-2022", "1-2022")

In [8]:
df.periodo.unique()

array(['1-2022', '2-2022'], dtype=object)

- Se descarta points_possible del analisis dado que los puntos posibles en casi su totalidad son iguales a 100 y los restantes presentan inconsistencias 

In [9]:
diferente_100 = df[(df["points_possible"] != 100) & (df["points_possible"].notnull())]
diferente_100.points_possible.describe(),diferente_100.score.describe()

(count    107.000000
 mean       0.934579
 std        2.924429
 min        0.000000
 25%        0.000000
 50%        0.000000
 75%        0.000000
 max       10.000000
 Name: points_possible, dtype: float64,
 count     32.000000
 mean      83.671875
 std       14.793015
 min       33.333333
 25%       74.270833
 50%       85.416667
 75%       95.625000
 max      100.000000
 Name: score, dtype: float64)

- La columna s_created_at presenta un problema con la fecha (2022-08-01 22:27:07) la cual representa casi la totalidad de este dato, se asume como un error y se omite

In [10]:
df["s_created_at"].isnull().sum()

np.int64(171918)

In [11]:
df.groupby("s_created_at").size()

s_created_at
2022-08-01 22:27:07    24189
2022-08-26 10:15:56        9
2022-10-07 11:54:20        7
2022-10-17 15:21:56        1
dtype: int64

Para mayor organizacion se difurca el dataset en dos partes
- df_estudiantes: Agrupa información relacionada directamente con el estudiante y su desempeño en el curso (notas, exámenes, datos del curso)

In [12]:
df_estudiantes = df[["user_uuid","course_uuid","legajo","course_name","periodo","particion",
                     "fecha_mesa_epoch","nombre_examen","nota_parcial","nota_final_materia"]].dropna()
df_estudiantes

,user_uuid,course_uuid,legajo,course_name,periodo,particion,fecha_mesa_epoch,nombre_examen,nota_parcial,nota_final_materia
29,13df535e-065c-4593-98ea-5b1e29015b7d,09614210-fce2-48bc-93e3-bc4bd441fe00,834066QFF,Progressive homogeneous structure,1-2022,28,2022-05-06,PRIMER PARCIAL(20),8.0,9.0
41,13df535e-065c-4593-98ea-5b1e29015b7d,09614210-fce2-48bc-93e3-bc4bd441fe00,834066QFF,Progressive homogeneous structure,1-2022,38,2022-05-31,SEGUNDO PARCIAL(20),10.0,9.0
84,8289fbcb-a999-4b5b-8d7e-a4ea0fe477d5,14d11dfe-01d7-4c8a-8489-59cce7e2d051,730281VXM,Switchable radical service-desk,1-2022,21,2022-04-01,PRIMER PARCIAL(20),4.0,5.0
108,8289fbcb-a999-4b5b-8d7e-a4ea0fe477d5,14d11dfe-01d7-4c8a-8489-59cce7e2d051,730281VXM,Switchable radical service-desk,1-2022,45,2022-05-10,SEGUNDO PARCIAL(20),5.0,5.0
109,8289fbcb-a999-4b5b-8d7e-a4ea0fe477d5,14d11dfe-01d7-4c8a-8489-59cce7e2d051,730281VXM,Switchable radical service-desk,1-2022,45,2022-05-10,SEGUNDO PARCIAL(20),5.0,5.0
...,...,...,...,...,...,...,...,...,...,...
195909,0760bfca-c724-46d1-bb60-c55c6d8a09f7,0034afe6-e996-4c26-b0b9-24dbb9535465,777338EQM,Stand-alone upward-trending secured line,1-2022,49,2022-07-07,SEGUNDO PARCIAL(20),6.0,7.0
195959,11c9300b-f47e-494b-9acf-0dd31ef72d35,298bc945-0162-479c-8d49-ed6f6530e52d,042355YYY,Multi-lateral modular algorithm,1-2022,32,2022-04-19,PRIMER PARCIAL(20),5.0,6.0
195969,11c9300b-f47e-494b-9acf-0dd31ef72d35,298bc945-0162-479c-8d49-ed6f6530e52d,042355YYY,Multi-lateral modular algorithm,1-2022,37,2022-04-28,SEGUNDO PARCIAL(20),6.0,6.0
196082,1f043fbc-2e51-4639-b99a-00e96f86968d,d9cc0ef0-3282-4c10-b2c7-bc231a26ca6a,398800TRX,Customizable hybrid forecast,1-2022,19,2022-04-11,PRIMER PARCIAL(20),10.0,10.0


- Se crea una columna binaria que indique si el estudiante aprobo o no el examen

In [13]:
df_estudiantes["aprobado"] = df_estudiantes["nota_final_materia"].apply(lambda x: 1 if x >= 5 else 0)

- Se crea la columna rango_nota que categoriza las notas finales en rangos ("bajo", "medio", "alto") para facilitar el análisis

In [14]:
bins = [0, 4, 7, 11]
labels = ["bajo", "medio", "alto"]
df_estudiantes["rango_nota"] = pd.cut(df_estudiantes["nota_final_materia"], bins=bins, labels=labels, right=False)

- Se crea la columna diferencia_notas que calcula la diferencia entre nota_final_materia y nota_parcial. Pudiendo identificar si los estudiantes mejoran o empeoran con el tiempo

In [15]:
df_estudiantes["diferencia_notas"] = df_estudiantes["nota_final_materia"] - df_estudiantes["nota_parcial"]

-  Se crea la columna promedio_nota_parcial que calcula el promedio de notas parciales por estudiante y curso

In [16]:
df_estudiantes["promedio_nota_parcial"] = df_estudiantes.groupby(["user_uuid", "course_uuid"])["nota_parcial"].transform("mean").round(2)

-  Se crea la columna std_nota_parcial que representa la desviación estándar de las notas parciales por estudiante y curso

In [17]:
df_estudiantes["std_nota_parcial"] = df_estudiantes.groupby(["user_uuid", "course_uuid"])["nota_parcial"].transform("std").round(2)
df_estudiantes["std_nota_parcial"] = df_estudiantes["std_nota_parcial"].fillna(0.)

In [18]:
df_estudiantes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7010 entries, 29 to 196100
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   user_uuid              7010 non-null   object        
 1   course_uuid            7010 non-null   object        
 2   legajo                 7010 non-null   object        
 3   course_name            7010 non-null   object        
 4   periodo                7010 non-null   object        
 5   particion              7010 non-null   int64         
 6   fecha_mesa_epoch       7010 non-null   datetime64[ns]
 7   nombre_examen          7010 non-null   object        
 8   nota_parcial           7010 non-null   float64       
 9   nota_final_materia     7010 non-null   float64       
 10  aprobado               7010 non-null   int64         
 11  rango_nota             7010 non-null   category      
 12  diferencia_notas       7010 non-null   float64       
 13  prome

Otras columnas utiles para df_estudiantes necesitan datos de df_entregas por lo que se crearan despues de este ultimo

- df_entregas: Contiene detalles específicos sobre las entregas de tareas (fechas, puntajes, tipo de entrega), que son un aspecto distinto del rendimiento académico

In [19]:
df_entregas = df[["user_uuid","course_uuid","legajo","course_name","periodo","particion","assignment_id",
                  "ass_name","ass_name_sub","sub_uuid","submission_type","ass_created_at","ass_due_at",
                  "ass_unlock_at","ass_lock_at","s_submitted_at","s_graded_at","score"]] 
df_entregas = df_entregas.loc[~df_entregas.iloc[:, 6:].isnull().all(axis=1)] # eliminamos filas en donde no existe informacion

Para calcular los datos faltantes de la columna ass_created_at se propone lo siguiente:
- Si agrupamos por course_uuid y calculamos cual es el valor de ass_created_at en donde si existe informacion podemos reemplazar los valores faltantes de ass_created_at por estos
- En el caso de ni siquiera contar con esa informacion se reemplaza con lo mas cercano, el valor de s_submitted_at

In [20]:
def get_mode(series):
    mode = pd.Series.mode(series)
    if len(mode) > 0:
        return mode.iloc[0]
    else:
        return np.nan

most_frequent_due_dates = df_entregas.groupby("course_uuid")["ass_created_at"].agg(get_mode)
fill_values = most_frequent_due_dates.to_dict()

df_entregas["ass_created_at"] = df_entregas.apply(
    lambda row: fill_values.get(row["course_uuid"], row["ass_created_at"]), axis=1
)


df_entregas["ass_created_at"] = df_entregas["ass_created_at"].fillna(df_entregas["s_submitted_at"])

- La columna ass_name gran cantidad de datos faltantes, pero tenemos la ventaja de contar con un plan B (ass_name_sub) el cual nos permite tener una aproximacion para completar el valor de ass_name. Esto facilita posteriores analisis

In [21]:
df_entregas["ass_name"] = df_entregas["ass_name"].fillna(df_entregas["ass_name_sub"])

Para calcular los datos faltantes de las columnas ass_due_at, ass_unlock_at, ass_lock_at se propone lo siguiente:
- Si agrupamos por course_uuid y ass_name podemos obtener los valores de columnas ass_due_at, ass_unlock_at, ass_lock_at en donde existen para ass_name (variable fill_values)
- Con fill_values podemos reemplazar los datos faltantes en las columnas ass_due_at, ass_unlock_at, ass_lock_at y tener una muy buena aproximacion

In [22]:
fill_values = {}
for col in ["ass_due_at", "ass_unlock_at", "ass_lock_at"]:
    most_frequent_values = (
        df_entregas.groupby(["course_uuid", "ass_name"])[col]
        .agg(get_mode)
        .reset_index()
    )

    fill_values[col] = most_frequent_values.set_index(
        ["course_uuid", "ass_name"]
    )[col].to_dict()


for col in ["ass_due_at", "ass_unlock_at", "ass_lock_at"]:
    df_entregas[col] = df_entregas.apply(
        lambda row: fill_values[col].get(
            (row["course_uuid"], row["ass_name"]), row[col]
        ),
        axis=1,
    )

- Calculamos el promedio del puntaje obtenido por estudiante en todas las entregas

In [23]:
df_entregas["promedio_score"] = df_entregas.groupby(["user_uuid"])["score"].transform("mean").round(2)

- Desviación estandar del puntaje obtenido por estudiante en todas las entregas

In [24]:
df_entregas["std_score"] = df_entregas.groupby(["user_uuid"])["score"].transform("std").round(2)

- Suma total de puntajes obtenidos por estudiante en todas las entregas

In [25]:
df_entregas["total_score"] = df_entregas.groupby(["user_uuid"])["score"].transform("sum").round(2)

- Agregamos una columna adicional (tiempo_entrega) que calcula el tiempo transcurrido entre ass_unlock_at y s_submitted_at. Para saber que tiempo dedicaron los estudiantes a cada tarea

In [26]:
df_entregas["tiempo_entrega"] = ((df_entregas["s_submitted_at"] - df_entregas["ass_unlock_at"]).dt.total_seconds() / 3600).round(1)

- Agregamos una columna adicional (tiempo_calificacion) calcula el tiempo entre s_submitted_at y s_graded_at. Para entender los tiempos de respuesta de los profesores

In [27]:
df_entregas["tiempo_calificacion"] = ((df_entregas["s_graded_at"] - df_entregas["s_submitted_at"]).dt.total_seconds() / 3600).round(1)

- Agregamos una columna adicional (dias_antes_vencimiento) representa la diferencia en dias entre s_submitted_at y ass_due_at. Un valor negativo indica que la tarea se entrego despues de la fecha limite

In [28]:
df_entregas["dias_antes_vencimiento"] = (df_entregas["ass_due_at"] - df_entregas["s_submitted_at"]).dt.days

- Agregamos una columna adicional (on_time) para determinar si el alumno subio las tareas a tiempo

In [29]:
df_entregas.loc[(df_entregas["s_submitted_at"].notnull()) & (df_entregas["s_graded_at"].isnull()), "s_graded_at"] = "calificacion_pendiente"
def determinar_estado(row):
    if pd.isna(row["s_submitted_at"]):
        return "no_entregado"
    elif row["s_submitted_at"] <= row["ass_due_at"]:
        return "a_tiempo"
    else:
        return "tarde"

df_entregas["on_time"] = df_entregas.apply(determinar_estado, axis=1)
df_entregas.loc[(df_entregas["s_submitted_at"].notna()) & (df_entregas["ass_due_at"].isna()), "on_time"] = "error"
df_entregas.loc[(df_entregas["ass_due_at"].isna()) & (df_entregas["s_graded_at"] == "calificacion_pendiente"), "on_time"] = "calificacion_pendiente"

C:\Users\tomas\AppData\Local\Temp\ipykernel_19160\3118674877.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'calificacion_pendiente' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df_entregas.loc[(df_entregas["s_submitted_at"].notnull()) & (df_entregas["s_graded_at"].isnull()), "s_graded_at"] = "calificacion_pendiente"


- Dataset de entregas relustante:

In [30]:
df_entregas

,user_uuid,course_uuid,legajo,course_name,periodo,particion,assignment_id,ass_name,ass_name_sub,sub_uuid,...,s_submitted_at,s_graded_at,score,promedio_score,std_score,total_score,tiempo_entrega,tiempo_calificacion,dias_antes_vencimiento,on_time
15,13df535e-065c-4593-98ea-5b1e29015b7d,09614210-fce2-48bc-93e3-bc4bd441fe00,834066QFF,Progressive homogeneous structure,1-2022,15,NaN,Actividad Práctica Integradora 1 [API1],Actividad Práctica Integradora 1 [API1],ef6fc92b-98da-40de-b440-1990379ee5e1,...,2022-04-03 03:19:59,2022-04-03 03:29:45,100.0,87.71,22.05,3069.85,312.3,0.2,7.0,a_tiempo
18,13df535e-065c-4593-98ea-5b1e29015b7d,09614210-fce2-48bc-93e3-bc4bd441fe00,834066QFF,Progressive homogeneous structure,1-2022,18,195289.0,Actividad Práctica Integradora 1 [API1],NaN,NaN,...,NaT,NaT,NaN,87.71,22.05,3069.85,NaN,NaN,NaN,no_entregado
21,13df535e-065c-4593-98ea-5b1e29015b7d,09614210-fce2-48bc-93e3-bc4bd441fe00,834066QFF,Progressive homogeneous structure,1-2022,21,195293.0,Trabajo Práctico 1 [TP1],Trabajo Práctico 1 [TP1],8891a347-bd41-4e5b-8c4b-cfea97c52985,...,2022-04-18 19:56:31,2022-04-25 00:55:57,70.0,87.71,22.05,3069.85,688.9,149.0,0.0,a_tiempo
22,13df535e-065c-4593-98ea-5b1e29015b7d,09614210-fce2-48bc-93e3-bc4bd441fe00,834066QFF,Progressive homogeneous structure,1-2022,21,195293.0,Trabajo Práctico 1 [TP1],Actividad Práctica Integradora 2 [API2],3de31206-90b2-45ee-b40f-f45ee35b3384,...,2022-04-18 01:09:33,2022-04-18 01:12:23,100.0,87.71,22.05,3069.85,670.2,0.0,1.0,a_tiempo
25,13df535e-065c-4593-98ea-5b1e29015b7d,09614210-fce2-48bc-93e3-bc4bd441fe00,834066QFF,Progressive homogeneous structure,1-2022,24,195290.0,Actividad Práctica Integradora 2 [API2],NaN,NaN,...,NaT,NaT,NaN,87.71,22.05,3069.85,NaN,NaN,NaN,no_entregado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196093,1f043fbc-2e51-4639-b99a-00e96f86968d,d9cc0ef0-3282-4c10-b2c7-bc231a26ca6a,398800TRX,Customizable hybrid forecast,1-2022,30,205725.0,Trabajo Práctico 2 [TP2],NaN,NaN,...,NaT,NaT,NaN,89.22,15.08,6156.02,NaN,NaN,NaN,no_entregado
196094,1f043fbc-2e51-4639-b99a-00e96f86968d,d9cc0ef0-3282-4c10-b2c7-bc231a26ca6a,398800TRX,Customizable hybrid forecast,1-2022,30,205726.0,Trabajo Práctico 3 [TP3],NaN,NaN,...,NaT,NaT,NaN,89.22,15.08,6156.02,NaN,NaN,NaN,no_entregado
196099,1f043fbc-2e51-4639-b99a-00e96f86968d,d9cc0ef0-3282-4c10-b2c7-bc231a26ca6a,398800TRX,Customizable hybrid forecast,1-2022,35,205722.0,Actividad Práctica Integradora 3 [API3],NaN,NaN,...,NaT,NaT,NaN,89.22,15.08,6156.02,NaN,NaN,NaN,no_entregado
196110,1f043fbc-2e51-4639-b99a-00e96f86968d,d9cc0ef0-3282-4c10-b2c7-bc231a26ca6a,398800TRX,Customizable hybrid forecast,1-2022,46,205723.0,Actividad Práctica Integradora 4 [API4],NaN,NaN,...,NaT,NaT,NaN,89.22,15.08,6156.02,NaN,NaN,NaN,no_entregado


Con las transformaciones realizadas en df_entregas podemos agregar nueva informacion a df_estudiantes!
- Numero de actividades realizadas por partición

In [31]:
df_actividades = df_entregas.groupby(["user_uuid", "course_uuid", "particion"]).size().reset_index(name="num_actividades")
df_estudiantes = df_estudiantes.merge(df_actividades, on=["user_uuid", "course_uuid", "particion"], how="left")
df_estudiantes["num_actividades"] = df_estudiantes["num_actividades"].fillna(0.)

- Puntaje total de tareas entregadas por estudiante y curso

In [32]:
df_puntaje_total = df_entregas.groupby(["user_uuid", "course_uuid"])["score"].sum().reset_index(name="puntaje_total").round(2)
df_estudiantes = df_estudiantes.merge(df_puntaje_total, on=["user_uuid", "course_uuid"], how="left")

- Numero de tareas entregadas por estudiante y curso

In [33]:
df_num_tareas = df_entregas.groupby(["user_uuid", "course_uuid"])["ass_name"].nunique().reset_index(name="num_tareas")
df_estudiantes = df_estudiantes.merge(df_num_tareas, on=["user_uuid", "course_uuid"], how="left")

- Promedio de puntajes de tareas entregadas por estudiante y curso

In [34]:
df_promedio_puntaje = df_entregas.groupby(["user_uuid", "course_uuid"])["score"].mean().reset_index(name="mean_puntaje_tareas").round(2)
df_estudiantes = df_estudiantes.merge(df_promedio_puntaje, on=["user_uuid", "course_uuid"], how="left")

- Promedio de la cantidad de dias desde la creación de la tarea hasta su vencimiento

In [35]:
df_dias_vencimiento = df_entregas.groupby(["user_uuid", "course_uuid"])["dias_antes_vencimiento"].mean().reset_index(name="dias_mean_antes_vencimiento").round(2)
df_estudiantes = df_estudiantes.merge(df_dias_vencimiento, on=["user_uuid", "course_uuid"], how="left")

- Dataset de estudiantes relustante:

In [36]:
df_estudiantes

,user_uuid,course_uuid,legajo,course_name,periodo,particion,fecha_mesa_epoch,nombre_examen,nota_parcial,nota_final_materia,aprobado,rango_nota,diferencia_notas,promedio_nota_parcial,std_nota_parcial,num_actividades,puntaje_total,num_tareas,mean_puntaje_tareas,dias_mean_antes_vencimiento
0,13df535e-065c-4593-98ea-5b1e29015b7d,09614210-fce2-48bc-93e3-bc4bd441fe00,834066QFF,Progressive homogeneous structure,1-2022,28,2022-05-06,PRIMER PARCIAL(20),8.0,9.0,1,alto,1.0,9.00,1.41,0.0,575.00,8,82.14,1.71
1,13df535e-065c-4593-98ea-5b1e29015b7d,09614210-fce2-48bc-93e3-bc4bd441fe00,834066QFF,Progressive homogeneous structure,1-2022,38,2022-05-31,SEGUNDO PARCIAL(20),10.0,9.0,1,alto,-1.0,9.00,1.41,1.0,575.00,8,82.14,1.71
2,8289fbcb-a999-4b5b-8d7e-a4ea0fe477d5,14d11dfe-01d7-4c8a-8489-59cce7e2d051,730281VXM,Switchable radical service-desk,1-2022,21,2022-04-01,PRIMER PARCIAL(20),4.0,5.0,1,medio,1.0,4.75,0.50,0.0,557.50,8,79.64,25.00
3,8289fbcb-a999-4b5b-8d7e-a4ea0fe477d5,14d11dfe-01d7-4c8a-8489-59cce7e2d051,730281VXM,Switchable radical service-desk,1-2022,45,2022-05-10,SEGUNDO PARCIAL(20),5.0,5.0,1,medio,0.0,4.75,0.50,2.0,557.50,8,79.64,25.00
4,8289fbcb-a999-4b5b-8d7e-a4ea0fe477d5,14d11dfe-01d7-4c8a-8489-59cce7e2d051,730281VXM,Switchable radical service-desk,1-2022,45,2022-05-10,SEGUNDO PARCIAL(20),5.0,5.0,1,medio,0.0,4.75,0.50,2.0,557.50,8,79.64,25.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7005,0760bfca-c724-46d1-bb60-c55c6d8a09f7,0034afe6-e996-4c26-b0b9-24dbb9535465,777338EQM,Stand-alone upward-trending secured line,1-2022,49,2022-07-07,SEGUNDO PARCIAL(20),6.0,7.0,1,alto,1.0,6.50,0.71,0.0,761.82,8,76.18,6.30
7006,11c9300b-f47e-494b-9acf-0dd31ef72d35,298bc945-0162-479c-8d49-ed6f6530e52d,042355YYY,Multi-lateral modular algorithm,1-2022,32,2022-04-19,PRIMER PARCIAL(20),5.0,6.0,1,medio,1.0,5.50,0.71,0.0,695.83,8,86.98,20.38
7007,11c9300b-f47e-494b-9acf-0dd31ef72d35,298bc945-0162-479c-8d49-ed6f6530e52d,042355YYY,Multi-lateral modular algorithm,1-2022,37,2022-04-28,SEGUNDO PARCIAL(20),6.0,6.0,1,medio,0.0,5.50,0.71,0.0,695.83,8,86.98,20.38
7008,1f043fbc-2e51-4639-b99a-00e96f86968d,d9cc0ef0-3282-4c10-b2c7-bc231a26ca6a,398800TRX,Customizable hybrid forecast,1-2022,19,2022-04-11,PRIMER PARCIAL(20),10.0,10.0,1,alto,0.0,9.50,0.71,1.0,594.20,8,84.89,21.86


In [37]:
# guardamos para futuros analisis 
df_estudiantes.to_csv("../data/df_estudiantes.csv", index=False)

In [38]:
# guardamos para futuros analisis 
df_entregas.to_csv("../data/df_entregas.csv", index=False)

Luego de este preprocessed estamos en un mejor punto de partida para realizar un EDA y posterior modelado